# Week 5 Homework - Solutions

This week's homework is focused on making further improvements to our COMPAS-like deep neural network. We will be performing hyperparameter tuning. As discussed during lecture, you might not always see huge improvements from regularization or dropout. Oftentimes, this is because **hyperparameters** including regularization and dropout strengths are not optimal. Hyperparameter tuning trains several different models, each with a different combinations of hyperparameter values.

## Setting up the Environment

You should have already cloned the `unit3` repository from GitHub and installed dependencies in requirements.txt [(instructions here, just in case)](http://web.stanford.edu/class/cs21si/setup.html) during class.

Run any code below by highlighting it and hitting `Shift + Enter`. Import the libraries below.

In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras import regularizers
from keras import backend as K
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import random

# fix random seed for reproducibility
np.random.seed(1337)

## Read and Clean in Data

In [ ]:
filename = "resources/compas-scores.csv"

# load data
data = pd.read_csv(filename, header = 0)

# select fields we want
fields_of_interest = ['name', 'sex', 'age', 'race', 'priors_count', 'c_charge_desc', 
                      'v_decile_score', 'decile_score', 'is_violent_recid', 'is_recid']
data = data[fields_of_interest]
data.columns = ['name', 'sex', 'age', 'race', 'num_priors', 'charge', 
                'violence_score', 'recidivism_score', 'violence_true', 'recidivism_true']

# remove records with missing scores
data = data.loc[(data.violence_score != -1) & (data.recidivism_score != -1)]
data = data.loc[(data.violence_true != -1) & (data.recidivism_true != -1)]

# convert strings to numerical values
races = ['African-American', 'Asian', 'Caucasian', 'Hispanic', 'Native American', 'Other']
sex_classes = {'Male': 0, 'Female' : 1}
race_classes = {races[i]: i for i in range(len(races))}

# 'Other': 0, 'Caucasian': 1, 'African-American': 2, 'Hispanic': 3, 'Asian': 4, 'Native American': 5}
data['sex'] = data['sex'].apply(lambda x: sex_classes[x])
data['race'] = data['race'].apply(lambda x: race_classes[x])
# threshold and binarize scores
data['violence_score'] = data['violence_score'].apply(lambda x: 0 if x <= 5 else 1)
data['recidivism_score'] = data['recidivism_score'].apply(lambda x: 0 if x <= 5 else 1)
print(data)

# convert pandas dataframe to numpy array for easier processing
data = data.values

## Partition into Train and Test Sets

This was all code you wrote during lecture, so we've given it to you as a freebie here!

In [ ]:
# split into input (X) and output (Y) variables
X = data[:,1:5] # sex, age, race, num_priors
y = data[:,7] # recidivism_score

num_train = int(math.ceil(X.shape[0]*0.8))
num_test = int(math.floor(X.shape[0]*0.2))

#########################################################
# Returns the specified records of a given array, from
# row_start to row_start + num_rows - 1 (inclusive).
#########################################################
def get_rows(dataset, row_start, num_rows):
    return dataset[row_start:row_start + num_rows]

X_train = get_rows(X, 0, num_train)
y_train = get_rows(y, 0, num_train)

X_test = get_rows(X, num_train, num_test)
y_test = get_rows(y, num_train, num_test)

num_classes = 2
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(X_train.shape[0], 'records in train set')
print(X_test.shape[0], 'records in test set')
print(X.shape[0], 'records in total')

## Set up our Evaluation Pipeline

Again, we wrote this together in class, so it's given to you here.

In [ ]:
#########################################################
# Trains and evaluates given model. Returns loss and 
# accuracy.
#########################################################
def eval(model, verb = 2):
    # fit the model
    model.fit(X_train, y_train, 
              epochs = epochs, 
              batch_size = batch_size,          
              validation_split = 0.1,
              verbose = verb,
              shuffle = False)
    
    # evaluate the model
    scores = model.evaluate(X_test, y_test)
    
    return scores

## Jupyter Exercise 1: Create and Evaluate your Own Classifier

During lecture, you learned about adding `Dropout`, `Dense` layers (which are just fully-connected layers that have the parameter `kernel_regularizer`), and `BatchNormalization` to a Keras model. Check out this [guide](https://keras.io/getting-started/sequential-model-guide/) for some good examples in case you want a refresher.

**Your task:** Now that you've gotten your feet wet with Keras, in this exercise, you get to decide how you want to set up your model! Add layers of any size (paired with any activation functions) as you see fit, and if you want to experiment with other Keras add-ons that we haven't discussed before, then go for it. Similarly, if you elect not to use L2-regularization or dropout, then that's up to you too. Good luck!

In [ ]:
batch_size = 256
epochs = 15
num_classes = 2

learning_rate = 5e-2
reg_strength = 0.15
dropout_strength = 0.1

#########################################################
# Initializes neural network with dropout.
#########################################################
def nn_classifier(learning_rate, reg_strength, dropout_strength):
    # create model
    model = Sequential()

    # YOUR CODE HERE:
    model.add(Dense(50, input_dim = X.shape[1], activation = 'relu')) 
    model.add(Dense(100, activation = 'relu', kernel_regularizer=regularizers.l2(reg_strength))) 
    model.add(Dense(50, activation = 'relu', kernel_regularizer=regularizers.l2(reg_strength))) 
    model.add(Dense(num_classes, activation = 'softmax'))
    # END CODE
    
    # compile model
    sgd = keras.optimizers.SGD(lr = learning_rate)
    model.compile(loss = keras.losses.categorical_crossentropy, 
                  optimizer = sgd, metrics=['accuracy'])
    
    return model

# Evaluate your model
model = nn_classifier(learning_rate, reg_strength, dropout_strength)
loss, acc = eval(model, verb = 0)
print('\n\nTest loss:', loss)
print('Test accuracy:', acc)

## Jupyter Exercise 2: Hyperparameter Tuning

For the purposes of playing around with hyperparameter tuning and understanding the motivations behind it, we're going to be working with a naive implementation below.  

**Your task**: Tuning the hyperparameters and developing intuition for how they affect the final performance is a large part of using neural networks, so we want you to get a lot of practice. Below, you should experiment with different values of the various hyperparameters, including learning rate, regularization strength, and dropout strength. Your goal in this exercise is to get as good of a result on the COMPAS dataset as you can, with a fully-connected deep neural network. Feel free to change the model you initialized above in Exercise 1 as well. The starter code below is there to give you a naive implementation of hyperparameter tuning. You can change it as you wish. **Aim for at least 75% accuracy on the test set with your best model.** Good luck!

In [ ]:
def tune_hyperparams():
    best_model = (None, None, None)
    running_best_accuracy = 0

    learning_rate = [1e-12, 1e-10, 5e-8]
    reg_strength = [1e-2] 
    dropout_strength = [0.1]
    
    for lr in learning_rate:
        for reg in reg_strength:
            for drop in dropout_strength:
                model = nn_classifier(lr, reg, drop)
                model_loss, model_acc = eval(model)
        
                print('\n val_acc: {:f}, lr: {:f}, reg: {:f}, drop: {:f}\n'.format(
                        model_acc, lr, reg, drop))
        
                if model_acc > running_best_accuracy:
                    model_params = {"lr": lr, "reg": reg, "drop": drop}
                    best_model = (model, model_acc, model_params)
                    running_best_accuracy = model_acc
            
    return best_model
        
best_model = tune_hyperparams()
print("\n\nBest Model Performance: ", best_model[1])
print("Hyperparameters of Best Model: ", best_model[2])